### Generate Cambdrige Data

This script leverages the CBECI API to download fresh Bitcoin energy consumption estimates from the [Cambridge Centre for Alternative Finance](https://ccaf.io/cbeci/index) and output the best guess annualized data for all energy price assumptions into a single CSV.

The motivation for this script is that the Cambridge Bitcoin Electricity Consumption Index assumes a constant fixed energy cost of $0.05/kWh, but in the face of rising energy prices there is desire to modify the output of the index to be more reflective of dynamic energy prices.

The output of this script is designed to be paired with EIA energy price data to allow for an evolving adjustment of what level of electricity spend the Bitcoin Network can support.

- Author: [Data Always](dataalways.substack.com)
- Last Modified: November 15, 2022

In [1]:
import numpy as np
import pandas as pd

In [2]:
"""
The analysis in this repository is only concerned with the CBECI
`BEST GUESS`. To avoid polution of the .csv we drop all the 
other columns. 
"""
def drop_standard_columns(dataframe):
    return dataframe.drop([
        'Timestamp', 
        'power MAX, GW', 
        'power MIN, GW', 
        'power GUESS, GW', 
        'annualised consumption MAX, TWh',
        'annualised consumption MIN, TWh'
        ], 
        axis= 1
    )

In [3]:
# Create the base dataframe on which to merge others by looking at the $0.01 data
df = pd.read_csv(
    "https://ccaf.io/cbeci/api/v1.2.0/download/data?price=0.01", 
    skiprows=1
)

# Drop the unwanted columns
df = drop_standard_columns(df)

# Rename the columns
df.rename(
    columns={
        'Date and Time': 'date', 
        'annualised consumption GUESS, TWh': 0.01
    }, 
    inplace=True
)

In [4]:
# Download the data for all prices and then merge them into one dataframe
for elec_cost in np.linspace(0.02, 0.19, 18):
    elec_str = "{:.2f}".format(elec_cost)
    
    tmp_df = pd.read_csv(
        f"https://ccaf.io/cbeci/api/v1.2.0/download/data?price={elec_str}", 
        skiprows=1
    )
    
    tmp_df = drop_standard_columns(tmp_df)
    
    tmp_df.rename(
        columns={
            'Date and Time': 'date', 
            'annualised consumption GUESS, TWh': elec_str
        }, 
        inplace=True
    )

    # Merge the data into the main data frame
    df = df.merge(
        tmp_df, 
        how="inner", 
        on="date"
    )

In [5]:
df.tail()

,date,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19
4570,2023-01-21T00:00:00,156.498516,140.596797,127.817483,111.977183,102.313307,92.998736,86.069422,76.653047,73.139366,66.125271,59.023982,55.809618,56.522370,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4571,2023-01-22T00:00:00,156.585136,140.674616,130.717452,112.353019,102.840586,93.451752,86.594969,77.143837,73.314114,67.135675,59.929410,55.840508,55.840508,53.025336,57.618272,50.858529,48.843833,45.716266,42.366663
4572,2023-01-23T00:00:00,158.292214,142.208239,134.554626,113.902868,104.449083,94.884407,87.876934,79.018538,74.113378,69.015357,61.961638,56.449276,56.449276,54.324673,57.618272,50.858529,48.843833,45.716266,42.366663
4573,2023-01-24T00:00:00,154.102047,138.443832,132.896823,111.172867,102.111776,92.742537,85.858063,77.835749,72.506344,68.234560,61.629554,54.955003,54.955003,54.019060,57.618272,50.858529,48.843833,45.716266,42.366663
4574,2023-01-25T00:00:00,159.533717,143.323594,139.124429,115.423058,106.208270,96.901875,89.221906,81.632800,75.530843,71.940892,64.332354,57.905555,56.892014,56.259227,57.618272,50.858529,48.843833,45.716266,42.366663


In [6]:
# Export the data to a .csv file
df.to_csv(
    './datasets/cbeci-annualized-consumption.csv', 
    index = False
)